In [1]:
import pandas as pd
import copulogram as cp
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import scipy as sc
import tensorflow as tf
import tensorflow_probability as tfp
tfd=tfp.distributions
tfb=tfp.bijectors
from scipy import io
import time
import matplotlib.pyplot as plt
from sklearn import mixture
import joblib as jbl
import sys
import sklearn
import pandas as pd
import seaborn as sns
from pathlib import Path
from mixture_models import GMCM, GMC
import utils as utl
from scipy.stats import multivariate_normal
import rpy2
import scoringrules as sr
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import time

In [2]:
import random
random.seed(3110251)

In [3]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"

# Read the CSV file from the URL
df = pd.read_csv(url, header=None)

# Select the columns
df = df.iloc[:,[14,26,28,29]]
df.columns = ["14","26","28","29"]
data = df.astype('float32').to_numpy()

In [4]:
def introduce_missingness(df, missing_percentage=0.1):
    # Copy the DataFrame to avoid modifying the original
    df_missing = df.copy()
    # Calculate the number of missing values to introduce
    n_missing = int(missing_percentage * df.size)
    # Flatten the DataFrame indices
    flat_indices = [(row, col) for row in range(df.shape[0]) for col in range(df.shape[1])]
    # Select random indices for missing values
    missing_indices = np.random.choice(range(len(flat_indices)), n_missing, replace=False)
    
    for idx in missing_indices:
        i, j = flat_indices[idx]
        df_missing[i, j] = np.nan
        
    return df_missing
# df = your original DataFrame
df_missing = introduce_missingness(data, missing_percentage=0.1)
mask = ~np.all(np.isnan(df_missing), axis=1)

# Filter the array to remove rows that are entirely NaN
df_missing = df_missing[mask]
train_idx = np.where(~np.isnan(df_missing).any(axis=1))[0]
# Find rows with AT LEAST one missing value
test_idx = np.where(np.isnan(df_missing).any(axis=1))[0]
train_data = df_missing[train_idx,]         # train: no missing values
test_data_with_missing = df_missing[test_idx,]
test_data = data[test_idx,]

In [5]:
start = time.time()
n_comps=2
nsamps,ndims = train_data.shape
print(f'Number of samples = {nsamps}, Number of dimensions = {ndims}')
gmcm = GMCM(ndims)
gmcm.fit_dist_IFM(train_data,
                  n_comps=n_comps,
                  method="GMCM",
                 data_vld= None,# fixed number of components
                 batch_size=10,
                 max_iters=6001,
                 regularize=True,
                 init = 'gmm', #(init option: 'gmm' warm starts gmcm learning compared to 'random')
                 print_interval=1000)

Number of samples = 376, Number of dimensions = 4
Learning Marginals
Marginals learnt in 6.87 s.
@ Iter:0,                         Training error: -12.0,                         Validation error: nan,                         Time Elapsed: 6.8 s
@ Iter:1000,                         Training error: -12.6,                         Validation error: nan,                         Time Elapsed: 14.3 s
@ Iter:2000,                         Training error: -12.7,                         Validation error: nan,                         Time Elapsed: 21.5 s
@ Iter:3000,                         Training error: -13.2,                         Validation error: nan,                         Time Elapsed: 28.4 s
@ Iter:4000,                         Training error: -12.1,                         Validation error: nan,                         Time Elapsed: 35.6 s
@ Iter:5000,                         Training error: -12.1,                         Validation error: nan,                         Time Elapsed: 43

In [6]:
list_indexes = list(range(ndims)) ## depends on the problem
#list_indexes = [0,1]
logits,mus,covs,_ = utl.vec2gmm_params(gmcm.ndims,gmcm.ncomps,gmcm.gmc.params)
alphas = tf.math.softmax(logits)
#dim_remove = list(set(list(range(self.ndims)))-set(dim_list))
mus_new = tf.gather(mus,list_indexes, axis=1).numpy()
covs_new = tf.TensorArray(tf.float32,gmcm.ncomps)
for k in range(gmcm.ncomps):
    temp_mat = covs[k].numpy()
    covs_new = covs_new.write(k,temp_mat[np.ix_(list_indexes, list_indexes)])
covs_new = covs_new.stack().numpy()
marginal_covs = np.array([[matrix[j, j] for j in range(ndims)] for matrix in covs_new])

In [7]:
weights = []
means = []
covariances = []

# Iterate over the 5 elements in gmcm.marg_dists
for i in range(ndims):
    marginal = gmcm.marg_dists[i]["marginal"]
    weights.append(marginal.weights_)
    means.append(marginal.means_.flatten())  # Flatten means to get a 1D array
    covariances.append(marginal.covariances_.flatten())  # Flatten covariances to get a 1D array
max_length = max(len(arr) for arr in weights)

# Pad arrays with zeros to ensure consistent length
padded_arrays = [np.pad(arr, (0, max_length - len(arr)), mode='constant') for arr in  weights]

# Concatenate into a single NumPy array
weights = np.vstack(padded_arrays)

max_length = max(len(arr) for arr in means)

# Pad arrays with zeros to ensure consistent length
padded_arrays = [np.pad(arr, (0, max_length - len(arr)), mode='constant') for arr in  means]

# Concatenate into a single NumPy array
means = np.vstack(padded_arrays)

max_length = max(len(arr) for arr in covariances)

# Pad arrays with zeros to ensure consistent length
padded_arrays = [np.pad(arr, (0, max_length - len(arr)), mode='constant') for arr in  covariances]

# Concatenate into a single NumPy array
covariances = np.vstack(padded_arrays)
os.chdir('C:\\Users\\antoi\\Documents\\UNIBE\\Conditioning\\Paper conditioning\\experiments\\imputation')
np.savetxt("weights_matrix_output.csv", np.transpose(weights), delimiter=",")
np.savetxt("means_matrix_output.csv", np.transpose(means), delimiter=",")
np.savetxt("sds_matrix_output.csv", np.transpose(covariances), delimiter=",")

In [8]:
data_tst_uniform = gmcm.marg_bijector.inverse(test_data).numpy()
data_tst_latent = gmcm.gmc.distribution.bijector.inverse(data_tst_uniform).numpy()

In [9]:
from scipy.stats import multivariate_normal
import numpy as np

def precompute_matrices(mus_new, covs_new, dimension_joint, n_comps):
    precomputed_matrices = {}

    for j in range(n_comps):
        cov_matrix = covs_new[j]
        
        all_indices = np.arange(dimension_joint)
        
        for mask in range(1 << dimension_joint):  # Create patterns for missing/observed indices
            indexes_missing = [i for i in range(dimension_joint) if mask & (1 << i)]
            indexes_observed = list(set(all_indices) - set(indexes_missing))
            
            if len(indexes_missing) == 0:
                continue
            
            key = (j, tuple(indexes_missing), tuple(indexes_observed))
            
            mean_vector = mus_new[j]
            
            mean_mis = mean_vector[indexes_missing]
            mean_obs = mean_vector[indexes_observed]
            
            cov_mm = cov_matrix[np.ix_(indexes_missing, indexes_missing)]
            cov_mo = cov_matrix[np.ix_(indexes_missing, indexes_observed)]
            cov_oo = cov_matrix[np.ix_(indexes_observed, indexes_observed)]
            
            inv_cov_oo = np.linalg.inv(cov_oo)
            cond_cov = cov_mm - cov_mo @ inv_cov_oo @ cov_mo.T
            
            precomputed_matrices[key] = {'mean_mis': mean_mis, 'mean_obs': mean_obs, 
                                         'cov_mo': cov_mo, 'inv_cov_oo': inv_cov_oo, 'cov_oo':cov_oo,
                                         'cond_cov': cond_cov}
    
    return precomputed_matrices

def compute_conditional_samples(precomputed_matrices, data_tst_latent, test_data_with_missing, mus_new, alphas, n_comps, n_samples, size_test):
    list_results = []
    conditional_parameters = []
    for i in range(size_test):
        dimension_predict = np.isnan(test_data_with_missing[i,]).sum()
        
        if dimension_predict != 0:
            indexes_missing = np.where(np.isnan(test_data_with_missing[i,]))[0]
            all_indices = np.arange(len(test_data_with_missing[i,]))
            indexes_observed = list(set(all_indices) - set(indexes_missing))
            conditioning_value = data_tst_latent[i, indexes_observed]
            new_alphas = np.zeros(n_comps)

            for j in range(n_comps):
                key = (j, tuple(indexes_missing), tuple(indexes_observed))
                
                mean_obs = precomputed_matrices[key]['mean_obs']
                cov_obs = precomputed_matrices[key]['cov_oo']
                
                marginal = multivariate_normal.pdf(conditioning_value, mean=mean_obs, cov=cov_obs)
                
                joint = 0
                for k in range(n_comps):
                    key_k = (k, tuple(indexes_missing), tuple(indexes_observed))
                    mean_obs_k = precomputed_matrices[key_k]['mean_obs']
                    cov_obs_k = precomputed_matrices[key_k]['cov_oo']
                    
                    joint += alphas[k] * multivariate_normal.pdf(conditioning_value, mean=mean_obs_k, cov=cov_obs_k)
                
                if joint != 0:
                    new_alphas[j] = alphas[j] * (marginal / joint)
            
            new_alphas /= new_alphas.sum()
            
            risk_vector_gmcm = np.zeros((n_samples, dimension_predict))
            
            for n in range(n_samples):
                new_component = np.random.choice(n_comps, p=new_alphas)
                
                key = (new_component, tuple(indexes_missing), tuple(indexes_observed))
                params = precomputed_matrices[key]
                
                cond_mean = params['mean_mis'] + params['cov_mo'] @ params['inv_cov_oo'] @ (conditioning_value - params['mean_obs'])
                cond_cov = params['cond_cov']
                
                if cond_mean.size == 1:
                    result = np.random.normal(loc=cond_mean, scale=np.sqrt(cond_cov))
                else:
                    result = np.random.multivariate_normal(mean=cond_mean, cov=cond_cov)
                
                risk_vector_gmcm[n, :] = result
            
            list_results.append(risk_vector_gmcm)
            cond_params = {"means":mus_new,"stds":np.sqrt(marginal_covs),"weights":alphas.numpy(), "indexes":indexes_missing}
            conditional_parameters.append(cond_params)
    
    return (list_results,conditional_parameters)

# Example usage
precomputed_matrices = precompute_matrices(mus_new=mus_new, covs_new=covs_new, dimension_joint = ndims, n_comps=n_comps)
list_results,conditional_parameters = compute_conditional_samples(precomputed_matrices=precomputed_matrices, data_tst_latent=data_tst_latent, 
                                                                  test_data_with_missing=test_data_with_missing, mus_new=mus_new,
                                                                  alphas=alphas, n_comps=n_comps,n_samples = 1000, size_test= np.shape(test_data)[0])


In [10]:
import os
import numpy as np

base_dir = "C:\\Users\\antoi\\Documents\\UNIBE\\Conditioning\\Paper conditioning\\experiments\\imputation\\conditional_parameters"
# Example list of dictionaries


# Ensure base directory exists
os.makedirs(base_dir, exist_ok=True)

# Iterate over the list of dictionaries
for index, params in enumerate(conditional_parameters):
    # Create a subfolder for each element
    subfolder = os.path.join(base_dir, f'element_{index}')
    os.makedirs(subfolder, exist_ok=True)
    np.savetxt(os.path.join(subfolder, 'indexes.csv'), params['indexes'], delimiter=',')
    # Save each array as a CSV file
    np.savetxt(os.path.join(subfolder, 'means.csv'), params['means'], delimiter=',')
    np.savetxt(os.path.join(subfolder, 'stds.csv'), params['stds'], delimiter=',')
    np.savetxt(os.path.join(subfolder, 'weights.csv'), params['weights'], delimiter=',')

print("CSV files created successfully.")

CSV files created successfully.


In [11]:

os.chdir('C:\\Users\\antoi\\Documents\\UNIBE\\Conditioning\\Paper conditioning\\experiments\\imputation\\before transfo')
files = glob.glob('*.csv')
for f in files:
    os.remove(f)
for i, arr in enumerate(list_results):
    df = pd.DataFrame(arr)
    df.to_csv(f'array_{i}.csv', index=False)
end = time.time()
print(end-start)

83.3797779083252


Time for doing the inversion with quantiles in R is about 3.6 seconds

In [12]:
start = time.time()
size_test = np.shape(test_data_with_missing)[0]
def impute_datasets(test_data, folder_path, n_imputations=1000):
    # Pre-allocate the list for imputed datasets
    imputed_datasets = np.array([test_data.copy().values for _ in range(n_imputations)])

    # Iterate over each row in the test data
    for i, row in test_data.iterrows():
        # Load the matrix corresponding to the current row
        matrix_file = os.path.join(folder_path, f'matrix{i+1}.csv')
        matrix = pd.read_csv(matrix_file, header=0).values

        # Get the indices of missing values in the current row
        missing_indices = np.where(row.isna())[0]
        
        # Assign imputed values from the matrix directly
        imputed_datasets[:, i, missing_indices] = matrix[:, :len(missing_indices)]

    # Convert numpy arrays back to DataFrames
    imputed_datasets = [pd.DataFrame(dataset, columns=test_data.columns) for dataset in imputed_datasets]

    return imputed_datasets

# Example usage


folder_path = 'C:\\Users\\antoi\\Documents\\UNIBE\\Conditioning\\Paper conditioning\\experiments\\imputation\\after transfo quantiles'
imputed_datasets_quantiles = impute_datasets(pd.DataFrame(test_data_with_missing), folder_path)

organized_data_quantiles = [[] for _ in range(size_test)]

# Populate the organized_data list
for dataset in imputed_datasets_quantiles:
    for i in range(size_test):
        organized_data_quantiles[i].append(dataset.iloc[i,:])

# Convert each entry in organized_data to a NumPy array (n_imputations, n_dims)
for i in range(size_test):
    organized_data_quantiles[i] = np.array(organized_data_quantiles[i])
end = time.time()
total_time_formatting = end-start
print(total_time_formatting)

15.106997966766357


In [13]:
start_mice = time.time()
imputed_datasets = []
for i in range(1000):
    imp = IterativeImputer(max_iter=10, random_state=i,sample_posterior=True)
    imp.fit(df_missing)
    data = imp.transform(df_missing)
    imputed_datasets.append(data[test_idx,])
arrays = np.stack(imputed_datasets)  

n_rows = arrays.shape[1]

# Now, for each j, extract the j-th row across all matrices
result_mice = [arrays[:, j, :] for j in range(n_rows)]
end_mice = time.time()
print(end_mice-start_mice)

180.79723525047302


In [14]:
energy_score_mice = []
energy_score_quantiles= []
N = len(organized_data_quantiles)
for i in range(N):
    energy_score_mice.append(sr.energy_score(test_data[i],
    result_mice[i]))
    energy_score_quantiles.append(sr.energy_score(test_data[i],
    organized_data_quantiles[i]))

In [15]:
np.mean(energy_score_mice)

0.2867351

In [16]:
np.mean(energy_score_quantiles)

0.252016